## Optimal Clustering

군집화 문제는 객체 집합과 이들 객체 집합과 연관이 있는 특성들의 집합으로 구성된다. 목적은 특성을 사용해서 객체들을 그룹으로 분리하는 것이다.
여기서 그룹 내 유사도가 최대화되고, 그들 간 유사도는 최소화된다. 과제를 푸는 데 있어서 알고리즘을 도울 예제를 제공하지 않기 때문에, 많은 금융 문제들에 있어서 대다수는 비지도학습의 행태를 띄고 있다.
군집화 문제는 투자 프로세스의 모든 단계에서 금융에 자연스럽게 나타난다. 예를 들어, 분석가들은 현재 사건과 유사한 과거 사건을 찾을 것이며, 이는 사건의 수치적 분류 체계를 개발하는 것과 관련된다.
포트폴리오 매니저들은 종종 다양한 특성에 대해 증권을 군집화해 동료 기업 간의 상대 가치를 도출한다. 리스크 매니저들은 공통 특성을 공유하는 증권에 집중하는 것을 피하고자 한다.(분산투자) 
증권 집합에 영향을 주는 흐름을 이해하고, 상승세 또는 대규모 매도가 특정 증권에 특수한 것인지 또는 여러 증권에 의해 공유된 범주에 영향을 미치는 것인지를 결정하기 원할 것이다.
이들 문제를 해결하는 데 있어 상관관계 기반의 거리를 비롯한 다양한 거리 개념을 사용한다.

### 근접성 행렬

$N \times F$ 차수의 데이터 행렬 $X$를 고려하자. 
여기서 $N$은 샘플의 수이고, $F$는 특성의 수이다. $F$특성을 사용해 $N$개의 샘플 간의 $N \times N$행렬로 표현되는 근접성을 계산한다. 근접성 척도는 유사도 또는 비유사도를 가리킬 수 있다. 비유사도 척도가 거리 척도의 조건을 만족하는 것은 편리하나 반드시 필요하지는 않다. 근접성 행렬은 무방향성 그래프로 표현될 수 있다. 그래프 엣지의 가중치는 유사도 또는 비유사도의 함수이다. 그러면 군집화 문제는 그래프를 연결된 구성 요소로 분할하는 것과 동일하다. 근접성 형성할 때 입력을 표준화해 한 특성의 크기가 나머지를 지배하는 것을 방지하는 것은 좋은 아이디어이다.

### 군집화 종류

두 종류의 주요 군집화 알고리즘, 즉 분할과 계층적 군집화 알고리즘이 있다. 분할 기법은 객체를 한 수준에서 분할한다. 계층적 기법은 하나의 모든 샘플을 포함하는 군집을 최정상으로 하고, 개별 포인트의 단일점으로 이뤄진 군집을 바닥으로 하는 분할의 중첩된 시퀀스를 생성한다. 계층적 군집화 알고리즘은 분할적일 수도 있고, 응집형일 수도 있다. 계층적 트리의 성장을 제약함으로써 계층적 군집화로부터 분할 군집화를 도출할 수 있다. 그러나 일반적으로 분할 군집화로부터 계층적 군집화를 도출할 수는 없다.

군집화의 정의에 따라 여러 유형의 군집화 알고리즘을 구별할 수 있다. 이는 다음을 포함한다.

1. 연결성 : 이 군집화는 계층적 군집화와 같이 거리 연결성을 기반으로 한다. 금융에서의 예를 위해 Lopez de Prado(2016)을 참고하자.
2. 중심 : 이들 알고리즘은 k-means와 같이 벡터 양자화를 수행한다. 금융에서의 예를 위해서는 Lopez de Prado와 Lewis(2018)을 참고하자.
3. 분포 : 군집이 통계 분포를 이용해 형성된다
4. 밀도 : 이들 알고리즘은 데이터 공간의 연결되고 밀집된 공간을 탐색한다. 예는 DBSCAN과 OPTICS를 들 수 있다.
5. 하위 공간 : 군집들이 특성과 관측의 2차원 위에 모델링된다. 예는 양방향 군집화이다. 예를 들어, 증권과 기간의 부분 집합들 간의 유사도를 동시에 식별할 수 있다.

어떤 알고리즘은 유사도 척도를 입력으로 요구하고, 다른 알고리즘은 비유사도 척도를 입력으로 기대한다. 특정 알고리즘에 올바른 입력을 전달하는 것은 중요하다. 예를 들어, 계층적 알고리즘은 전형적으로 거리를 입력으로 취하고, 이는 이웃에 있는 아이템을 군집화할 것이다. 중심, 분포, 밀도 방법은 벡터 공간 좌표를 기대하며, 거리를 직접 다룬다. 그러나 거리 행렬을 바로 이용하는 양방향 군집화는 가장 거리가 먼 원소들을 군집화한다. 이를 수행하는 하나의 방법은 거리의 역수에 대해서 양방향 군집화하는 것이다.

만약 특성수가 관측수를 많이 초과한다면 차원의 저주는 군집화에 문제를 일으킨다. 관측을 생성하는 대부분의 공간이 빈 공간이어서 이떤 그룹을 식별하는 것도 어렵게 할 것이다. 이를 해결하는 하나의 방법은 PCA가 특성 수를 축소하는 것처럼 데이터 행렬 $X$를 저차원 공간에 투영하는 것이다. 또 하나의 해법은 근접성 행렬을 저차원 공간에 투영하고, 이를 새로운 $X$ 행렬로 사용하는 것이다. 두 경우 모두에 있어서, 잡음과 주음 제거는 신호에 연관된 차원 수를 식별하는 것을 돕는다.